### Adjust Word Level

In [85]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk import word_tokenize
from itertools import islice
import glob

In [86]:
lang_folder = "Turkish"
lang_pair = "Arabic"

In [87]:
word_start = 290
word_end = 300
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word = df_word.iloc[word_start:word_end,]
df_word.reset_index(drop=True, inplace=True)
df_word

,word,frequency
0,ha,257750
1,bakın,256830
2,dışarı,254689
3,mutlu,254555
4,ondan,252864
5,eminim,252546
6,insan,252056
7,gördüm,251150
8,bazı,249261
9,polis,247969


In [88]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence_All.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext.
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [89]:
df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence_All.csv")  # ext. sentence and ngram
df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext.
df_threegram_sent

,threegram,frequency
0,ne demek istiyorsun,55905
1,bu da ne,50098
2,hayır hayır hayır,41386
3,sen iyi misin,32965
4,çok teşekkür ederim,27663
...,...,...
1039403,kapıyı kapat aaron,5
1039404,kardeşime bir hediye,5
1039405,sonra da rachampsı,5
1039406,usta lee neredesin,5


In [90]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_Arabic (ara)_Etymologeek_Result_All.xlsx']

In [91]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,mı,mıknatıs,Turkish,(Noun) Magnet.Magnet.,مِغْنَاطِيس,Arabic (ara),NaN
1,mı,mısdak,Turkish,(Noun) (obsolete) criterion.(obsolete) criterion.,مِصْدَاق,Arabic (ara),NaN
2,şey,şey,Turkish,(Noun Interjection) (slang) genital organ. Thi...,شَيْئاً,Arabic (ara),NaN
3,şey,şeytan,Turkish,(Noun) Devil.Devil.,شَيَّطَ,Arabic (ara),NaN
4,şey,şeytanî,Turkish,(Adjective) Devilish.Devilish.,شَيْطَانِيّ,Arabic (ara),NaN
...,...,...,...,...,...,...,...
362,avra,avrat,Turkish,(Noun) Wife.Wife.,عَوِرَ,Arabic (ara),NaN
363,molla,molla,Turkish,NaN,مَوْلًى,Arabic (ara),NaN
364,ruba,rubai,Turkish,"(Noun) Rubai; quatrain in classical Arabic, Pe...",رباع,Arabic (ara),"Four at a time, in a group of four, in fours."
365,كتاب,كتاب,Turkish,NaN,كتاب,Arabic (ara),"(obsolete). Book. Letter, note, paper, piece o..."


In [92]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [93]:
ngram_list = []
for i in df_pair["dict_entry_main"]:
    for j in word_tokenize(i):
        for k in df_word["word"]:
            twogram_1_2 = f"{j} {k}"
            ngram_list.append(twogram_1_2)
            twogram_2_1 = f"{k} {j}"
            ngram_list.append(twogram_2_1)
df_pair_ngram = pd.DataFrame(ngram_list)
df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
df_pair_ngram.drop_duplicates(inplace=True)
df_pair_ngram.reset_index(drop=True, inplace=True)
df_pair_ngram

,twogram
0,mıknatıs ha
1,ha mıknatıs
2,mıknatıs bakın
3,bakın mıknatıs
4,mıknatıs dışarı
...,...
7350,gördüm حاضر
7351,حاضر bazı
7352,bazı حاضر
7353,حاضر polis


In [94]:
df_lang_pair_ngram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
df_lang_pair_ngram.rename(columns={"twogram":"twogram_pair"}, inplace=True)
df_lang_pair_ngram.drop_duplicates(inplace=True)
df_lang_pair_ngram

,twogram_pair,frequency
0,tamam bakın,583
1,ha tamam,411
2,elbette eminim,398
3,tamam gördüm,331
4,hayır bakın,299
...,...,...
185,ha hazır,3
186,ha hap,3
187,ani ha,3
188,ha millet,3


In [95]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np

In [96]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [97]:
word_list = df_word["word"].values.tolist()

In [98]:
data_kind = "twogram"

In [99]:
d_list  = df_twogram_sent.iloc[:,0].values.tolist()

In [100]:
resultlist = []

In [101]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(d_list):
    mergelist = []
    try:
        word = d_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(d_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list) # string_word liste

In [102]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result = df_result.rename(columns = {0: f"{data_kind}"})
df_result.iloc[:,0] = df_result.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
df_merge = pd.merge(df_result, df_twogram_sent, how="inner", on=f"{data_kind}")
df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
df_merge_result.drop_duplicates(inplace=True)
df_merge_result.reset_index(drop=True, inplace=True)
df_twogram_result = df_merge_result
df_twogram_result

,twogram,frequency
0,ondan eminim,117
1,dışarı bakın,54
2,polis ha,20
3,polis gördüm,13
4,ha dışarı,4
5,ha ondan,3


In [103]:
data_kind = "threegram"

In [104]:
d_list  = df_threegram_sent.iloc[:,0].values.tolist()

In [105]:
resultlist = []

In [106]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(d_list):
    mergelist = []
    try:
        word = d_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(d_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list) # string_word liste

In [107]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result = df_result.rename(columns = {0: f"{data_kind}"})
df_result.iloc[:,0] = df_result.iloc[:,0].apply(lambda x: remove_repetition(x))
df_merge = pd.merge(df_result, df_threegram_sent, how="inner", on=f"{data_kind}")
df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
df_merge_result.drop_duplicates(inplace=True)
df_merge_result.reset_index(drop=True, inplace=True)
df_threegram_result = df_merge_result
df_threegram_result

,threegram,frequency


In [108]:
pd.concat([df_word, df_lang_pair_ngram, df_twogram_result, df_threegram_result], axis=1)

,word,frequency,twogram_pair,frequency,twogram,frequency,threegram,frequency
0,ha,257750.0,tamam bakın,583,ondan eminim,117.0,NaN,NaN
1,bakın,256830.0,ha tamam,411,dışarı bakın,54.0,NaN,NaN
2,dışarı,254689.0,elbette eminim,398,polis ha,20.0,NaN,NaN
3,mutlu,254555.0,tamam gördüm,331,polis gördüm,13.0,NaN,NaN
4,ondan,252864.0,hayır bakın,299,ha dışarı,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...
185,NaN,NaN,ha hazır,3,NaN,NaN,NaN,NaN
186,NaN,NaN,ha hap,3,NaN,NaN,NaN,NaN
187,NaN,NaN,ani ha,3,NaN,NaN,NaN,NaN
188,NaN,NaN,ha millet,3,NaN,NaN,NaN,NaN


#### While Loop

In [119]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize

In [120]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [121]:
lang_folder = "Turkish"
lang_pair = "Arabic"

In [122]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
164294,buldugumuzda,43
164295,boşandıklarını,42
164296,endişenlenme,42
164297,uzaylıymışım,42


In [123]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence_All.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext.
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [124]:
df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence_All.csv")  # ext. sentence and ngram
df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext.
df_threegram_sent

,threegram,frequency
0,ne demek istiyorsun,55905
1,bu da ne,50098
2,hayır hayır hayır,41386
3,sen iyi misin,32965
4,çok teşekkür ederim,27663
...,...,...
1039403,kapıyı kapat aaron,5
1039404,kardeşime bir hediye,5
1039405,sonra da rachampsı,5
1039406,usta lee neredesin,5


In [125]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")

In [126]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,mı,mıknatıs,Turkish,(Noun) Magnet.Magnet.,مِغْنَاطِيس,Arabic (ara),NaN
1,mı,mısdak,Turkish,(Noun) (obsolete) criterion.(obsolete) criterion.,مِصْدَاق,Arabic (ara),NaN
2,şey,şey,Turkish,(Noun Interjection) (slang) genital organ. Thi...,شَيْئاً,Arabic (ara),NaN
3,şey,şeytan,Turkish,(Noun) Devil.Devil.,شَيَّطَ,Arabic (ara),NaN
4,şey,şeytanî,Turkish,(Adjective) Devilish.Devilish.,شَيْطَانِيّ,Arabic (ara),NaN
...,...,...,...,...,...,...,...
362,avra,avrat,Turkish,(Noun) Wife.Wife.,عَوِرَ,Arabic (ara),NaN
363,molla,molla,Turkish,NaN,مَوْلًى,Arabic (ara),NaN
364,ruba,rubai,Turkish,"(Noun) Rubai; quatrain in classical Arabic, Pe...",رباع,Arabic (ara),"Four at a time, in a group of four, in fours."
365,كتاب,كتاب,Turkish,NaN,كتاب,Arabic (ara),"(obsolete). Book. Letter, note, paper, piece o..."


In [127]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [128]:
# while loop code block
word_start = 0
word_end = 10
word_limit = 300
part_num = 1
while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)
    # language pair twogram
    ngram_list = []
    for i in df_pair["dict_entry_main"]:
        for j in word_tokenize(i):
            for k in df_word["word"]:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_ngram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_ngram.rename(columns={"twogram":"twogram_pair"}, inplace=True)
    df_lang_pair_ngram.drop_duplicates(inplace=True)

    # twogram
    word_list = df_word["word"].values.tolist()
    data_kind = "twogram"
    d_list  = df_twogram_sent.iloc[:,0].values.tolist()
    
    resultlist = []

    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste 

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list)
    df_result = df_result.rename(columns = {0: f"{data_kind}"})
    df_result.iloc[:,0] = df_result.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
    df_merge = pd.merge(df_result, df_twogram_sent, how="inner", on=f"{data_kind}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result.reset_index(drop=True, inplace=True)
    df_twogram_result = df_merge_result
    
    # threegram
    data_kind = "threegram"
    d_list  = df_threegram_sent.iloc[:,0].values.tolist()

    resultlist = []

    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list)
    df_result = df_result.rename(columns = {0: f"{data_kind}"})
    df_result.iloc[:,0] = df_result.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_merge = pd.merge(df_result, df_threegram_sent, how="inner", on=f"{data_kind}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result.reset_index(drop=True, inplace=True)
    df_threegram_result = df_merge_result

    df_output_result = pd.concat([df_word, df_lang_pair_ngram, df_twogram_result, df_threegram_result], axis=1)
    df_output_result.to_excel(f"Result_{part_num}.xlsx", index=False)

    word_start += 10
    word_end += 10
    part_num += 1     